In [ ]:
%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install folium --break-system-packages
%pip install ipywidgets --break-system-packages

# kernel restart required!!!

In [ ]:
import os
os.environ['AICORE_AUTH_URL'] = 'xxx'
os.environ['AICORE_CLIENT_ID'] = 'xxx'
os.environ['AICORE_CLIENT_SECRET'] = 'xxx'
os.environ['AICORE_RESOURCE_GROUP'] = 'default'
os.environ['AICORE_BASE_URL'] = 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com'

from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores.hanavector import HanaDB
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings, ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain_core.prompts import ChatPromptTemplate
from hdbcli import dbapi
import csv
import json

In [ ]:
# Define the schema using Pydantic v2
class OutputSchema(BaseModel):
    commodity_code: str = Field(description="The code representing the commodity")
    keywords: list[str] = Field(description="A list of keywords related to the commodity")

In [ ]:
embeddings = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

template = """ Objective: Create semantic embeddings to accurately map invoice line items to UNSPSC codes using keyword variations, 
based on the provided UNSPSC description.

Guidelines:  
1. UNSPSC Structure Analysis
- Analyze hierarchy: Segment > Family > Class > Commodity  
- Focus on commodity-level descriptions (most granular tier).  

2. Keyword Extraction
- Primary Term: Identify the product name or type (e.g., "tomatoes", "pasta").
- Secondary Term: Specify the product's state or packaging (e.g., "canned", "frozen", "glass jars").
- Generalization: Standardize specific terms to broader categories (e.g., "ginger thins → biscuit", "liquorice → candy").
- Exclusion: Remove non-essential words such as articles, generic verbs, or irrelevant descriptors.

3. Embedding Generation Rules
- Create 3 variants per code:  
  - Line 1: Original UNSPSC description keywords  
  - Lines 2-3: Semantic equivalents preserving core meaning  
- Avoid terms overlapping with other UNSPSC entries.  

Output Format: {format_instructions}

UNSPSC description: {UNSPSC}
"""

#prompt_template = ChatPromptTemplate.from_template(template)



In [ ]:
# Set up the JSON output parser with Pydantic v2 schema
parser = JsonOutputParser(pydantic_object=OutputSchema)

In [ ]:
# Define a function to query the LLM with vector-based context
def retrieve_and_query_llm(UNSPSC: str, k: int = 1) -> str:

    #prompt = prompt_template.format(UNSPSC=UNSPSC)
    # Query the LLM
    proxy_client = get_proxy_client('gen-ai-hub')
    model = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)

    # Create prompt template with format instructions
    prompt = PromptTemplate(
        template=template,
        input_variables=["UNSPSC"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Create and run the chain
    chain = prompt | model | parser

    # Example usage
    result = chain.invoke({"UNSPSC": UNSPSC})

    return result

In [ ]:
file_path = './data/'
file_in_name = 'Maersk_UNSPSC_50170000.csv'
file_out_name = 'Maersk_UNSPSC_5017000_Enriched.csv'

In [ ]:

with open(file_path + file_in_name, mode ='r') as infile, open(file_path + file_out_name, mode='w', encoding='utf-8', newline='\n') as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';', quoting=csv.QUOTE_NONE)
    for row_count, inline in enumerate(reader):
        if (row_count == 0):
            outline = ['Commodity Code; Commodity Keywords; Examples']
            cleaned_data = outline[0].strip("'").strip('"')
            cleaned_data = cleaned_data.split('\n')
            #print(cleaned_data)
            for row in cleaned_data:
                    #print('Row: ', row)
                    writer.writerow(row.split(';'))
        
        else:
            if (len(inline[11]) > 0):
                try:
                    commodity_code = int(inline[11])

                    if (commodity_code < 50199000):
                        print('Row Count: ', row_count, commodity_code)
                        response = retrieve_and_query_llm(str(inline))
                        #print('inline: ', inline, ' - ', response)
                        #print(response['commodity_code'], response['keywords'], type(response['keywords']))

                        for keyword in response['keywords']:
                            outline = str(response['commodity_code']) + ';' + str(keyword)
                            writer.writerow(outline.split(';'))

                except:
                    print('error')

        if (row_count == 10):
            break

print('FINISHED')



In [ ]:
from langchain_core.documents import Document

documents = []

with open(file_path + file_out_name, mode ='r') as infile:
    reader = csv.reader(infile, delimiter=';')
    for row_count, inline in enumerate(reader):
        if row_count > 0:
            print(inline)
            commodity_code = inline[0]
            commodity_keywords = inline[1]
            documents.append(
                Document(
                    page_content=commodity_keywords,
                    metadata={
                        "item_number": commodity_code
                    }
                )
            )
            
        #if row_count == 10:
        #    break

#print(docs)

In [ ]:
connection = dbapi.connect(
    address='xxx',
    port='443',
    user='xxx',
    password='xxx',
    autocommit=True,
    sslValidateCertificate=False,
)

db = HanaDB(
    embedding=embeddings,
    connection=connection,
    table_name="Maersk_Embeddings"
)

# Delete already existing documents from the table
db.delete(filter={})

#db.initialize()
db.add_documents(documents)

print('Finished')

In [ ]:
import pandas as pd

file_name = 'Maersk_InvoiceLines.csv'

# Read the CSV file with UTF-8 encoding
df = pd.read_csv(file_path + file_name, delimiter=';', encoding='utf-8')

print(df)

#df = df.sort_values(by=['DOCID', 'ITEMID'])
#df = df.drop_duplicates(subset=['SGTXT'], ignore_index = True)
print(df.shape[0])
print(df.columns)

In [ ]:
correct = 0

for index, row in df.iterrows():
    Invoice_Line = row['INVOICE_LINE']
    UNSPSC_predict_code = int(row['UNSPSC prediction'])
    UNSPSC_predict_label = row['UNSPSC prediction label']
    UNSPSC_predict_similarity = row['UNSPSC prediction similarity']

    print(index, Invoice_Line, ' - ', UNSPSC_predict_code, UNSPSC_predict_label, UNSPSC_predict_similarity) 
    similar_items = db.similarity_search_with_score(Invoice_Line, k=3)

    for item in similar_items:
        #print(item)
        commodity_code = item[0].metadata['item_number']
        commodity_label = str(item[0].page_content).strip()
        score = item[1]

        if (UNSPSC_predict_code == commodity_code):
            correct = correct+1

        print(commodity_code, commodity_label, score)

    print('Number of correct: ', correct)
    print()

    #if index == 100:
    #   break

print('Finished...')